**Need work..later**

In [ ]:
#import os
#import configparser
#from pyspark.sql import SparkSession

In [2]:
#config = configparser.ConfigParser()
#config.read('dl.cfg', encoding='utf-8-sig')

#os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
#os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
#spark = SparkSession.builder\
#                     .config("spark.jars.packages",
#                             "org.apache.hadoop:hadoop-aws:2.7.0")\
#                     .getOrCreate()

In [4]:
#song_data_path='s3a://udacity-dend/song_data/*/*/*/*.json'
#log_data_path='s3a://udacity-dend/log_data/*.json'

In [5]:
#song_data = spark.read.json(song_data_path)
#song_data.printSchema()

## Starting here

In [6]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, FloatType, TimestampType, LongType, DoubleType

In [7]:
from zipfile import ZipFile
import numpy as np
import pandas as pd

In [8]:
import datetime          
from pyspark.sql.functions import udf, col
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second

In [9]:
from pyspark.sql.functions import monotonically_increasing_id

In [10]:
pd.set_option('max_colwidth', 200)

In [11]:
# creating a spark session
spark = SparkSession \
               .builder \
               .appName("DataLake - Non_Cloud_Application") \
               .getOrCreate()

In [12]:
spark.sparkContext.getConf().getAll()

[('spark.app.name', 'DataLake - Non_Cloud_Application'),
 ('spark.driver.port', '38637'),
 ('spark.app.id', 'local-1643951914205'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '156dd3735337'),
 ('spark.ui.showConsoleProgress', 'true')]

In [13]:
#spark

In [14]:
# correct....here....

### Reading zip files from `local/current` directory location

In [15]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('data/log-data.zip', 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall('data/log-data/')
    
with ZipFile('data/song-data.zip', 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall('data/song-data/')

In [16]:
# setting log-data.json path and read log-data file first
path = "data/log-data"
log_df = spark.read.json(path)

In [17]:
# setting song_data path and read it with set data type
song_data_path = "data/song-data/song_data/*/*/*/*.json"
# song_data_path = "data/song-data/song_data/A/A/A/*.json"

# setting a schema for song_data columns
song_data_schema = StructType([
    StructField("artist_id", StringType()),
    StructField("artist_latitude", FloatType()),
    StructField("artist_longitude", FloatType()),
    StructField("artist_location", StringType()),
    StructField("artist_name", StringType()),
    StructField("song_id", StringType()),
    StructField("title", StringType()),
    StructField("duration", DoubleType()),
    StructField("year", IntegerType())
])
 # read song data file with schema asserted
song_df = spark.read.json(song_data_path, schema=song_data_schema)
song_df.toPandas().head(3)

,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,ARDR4AC1187FB371A1,NaN,NaN,,Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0
1,AREBBGV1187FB523D2,NaN,NaN,"Houston, TX",Mike Jones (Featuring CJ_ Mello & Lil' Bran),SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),173.66159,0
2,ARMAC4T1187FB3FA4C,40.826241,-74.47995,"Morris Plains, NJ",The Dillinger Escape Plan,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,207.77751,2004


In [18]:
log_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [19]:
song_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: float (nullable = true)
 |-- artist_longitude: float (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)



In [20]:
log_df.toPandas().head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26


In [21]:
song_df.toPandas().head(3)

,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,ARDR4AC1187FB371A1,NaN,NaN,,Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0
1,AREBBGV1187FB523D2,NaN,NaN,"Houston, TX",Mike Jones (Featuring CJ_ Mello & Lil' Bran),SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),173.66159,0
2,ARMAC4T1187FB3FA4C,40.826241,-74.47995,"Morris Plains, NJ",The Dillinger Escape Plan,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,207.77751,2004


In [22]:
log_df.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [23]:
song_df.columns

['artist_id',
 'artist_latitude',
 'artist_longitude',
 'artist_location',
 'artist_name',
 'song_id',
 'title',
 'duration',
 'year']

>**Keeping pages with "NextSong" only values**

In [24]:
log_df.filter(log_df.page == 'NextSong').count()

6820

In [25]:
log_df = log_df.filter(log_df.page == 'NextSong')

In [26]:
print('Song pages only = ', log_df.count())
log_df.select("page").dropDuplicates().show()

Song pages only =  6820
+--------+
|    page|
+--------+
|NextSong|
+--------+



> **ts column to timestamp conversion**

In [27]:
# ts/Timestamp conversion with 'udf' function from 'log-df' table
converting_ts = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0), TimestampType())
log_df = log_df.withColumn("start_time", converting_ts(log_df.ts))

In [28]:
log_df.select(['ts', 'start_time']).show(5)

+-------------+--------------------+
|           ts|          start_time|
+-------------+--------------------+
|1542241826796|2018-11-15 00:30:...|
|1542242481796|2018-11-15 00:41:...|
|1542242741796|2018-11-15 00:45:...|
|1542253449796|2018-11-15 03:44:...|
|1542260935796|2018-11-15 05:48:...|
+-------------+--------------------+
only showing top 5 rows



In [29]:
log_df.select('ts', 'start_time').printSchema()

root
 |-- ts: long (nullable = true)
 |-- start_time: timestamp (nullable = true)



In [30]:
# Creating timeStamp column into hour, day, week, month, year, weekday columns
log_df.select(['ts','start_time'
                  , dayofmonth("start_time").alias('day')
                  , month("start_time").alias('month')
                  , year("start_time").alias('year')
                  , hour("start_time").alias('hour')
                  , minute("start_time").alias('minute')
                  , second("start_time").alias('second')
                 ]).show(5)

+-------------+--------------------+---+-----+----+----+------+------+
|           ts|          start_time|day|month|year|hour|minute|second|
+-------------+--------------------+---+-----+----+----+------+------+
|1542241826796|2018-11-15 00:30:...| 15|   11|2018|   0|    30|    26|
|1542242481796|2018-11-15 00:41:...| 15|   11|2018|   0|    41|    21|
|1542242741796|2018-11-15 00:45:...| 15|   11|2018|   0|    45|    41|
|1542253449796|2018-11-15 03:44:...| 15|   11|2018|   3|    44|     9|
|1542260935796|2018-11-15 05:48:...| 15|   11|2018|   5|    48|    55|
+-------------+--------------------+---+-----+----+----+------+------+
only showing top 5 rows



In [31]:
log_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)



> **Inserting these new columns back to 'log_df' table**

In [32]:
# Displaying time_table into the format
log_df = log_df.withColumn('day', F.dayofmonth('start_time')) \
                      .withColumn('month', F.month('start_time')) \
                      .withColumn('year', F.year('start_time')) \
                      .withColumn('hour', F.hour('start_time')) \
                      .withColumn('minute', F.minute('start_time')) \
                      .withColumn('second', F.second('start_time')) \
                      .withColumn('week', F.weekofyear('start_time')) \
                      .withColumn('weekday', F.dayofweek('start_time')).dropDuplicates()

log_df.toPandas().head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,userId,start_time,day,month,year,hour,minute,second,week,weekday
0,Charttraxx Karaoke,Logged In,Aleena,F,27,Kirby,225.17506,paid,"Waterloo-Cedar Falls, IA",PUT,...,44,2018-11-15 19:42:36.796,15,11,2018,19,42,36,46,5
1,Harmonia,Logged In,Tegan,F,19,Levine,655.77751,paid,"Portland-South Portland, ME",PUT,...,80,2018-11-21 02:18:40.796,21,11,2018,2,18,40,47,4


In [33]:
log_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- second: integer (nullable = true)
 |-- week: integer (nullable = tr

### Creating `Dimension` & `Fact` tables

> **1. Time table**

In [34]:
#time_table = log_df.select(['ts','timeStamp']).withColumnRenamed('timeStamp','start_time') 

In [35]:
time_table = log_df.select(["ts", "start_time", "hour","minute","second", "day", "week", "month", "year", "weekday"])\
                                   .drop_duplicates()

In [36]:
time_table.toPandas().head(3)

,ts,start_time,hour,minute,second,day,week,month,year,weekday
0,1543596702796,2018-11-30 16:51:42.796,16,51,42,30,48,11,2018,6
1,1541622828796,2018-11-07 20:33:48.796,20,33,48,7,45,11,2018,4
2,1542147016796,2018-11-13 22:10:16.796,22,10,16,13,46,11,2018,3


In [37]:
# Displaying time_table into the format
#time_table = time_table.withColumn('day', F.dayofmonth('start_time')) \
#                      .withColumn('month', F.month('start_time')) \
#                      .withColumn('year', F.year('start_time')) \
#                      .withColumn('hour', F.hour('start_time')) \
#                      .withColumn('minute', F.minute('start_time')) \
#                      .withColumn('second', F.second('start_time')) \
#                      .withColumn('week', F.weekofyear('start_time')) \
#                      .withColumn('weekday', F.dayofweek('start_time')).dropDuplicates()

#time_table.show(5)

In [38]:
time_table.dtypes

[('ts', 'bigint'),
 ('start_time', 'timestamp'),
 ('hour', 'int'),
 ('minute', 'int'),
 ('second', 'int'),
 ('day', 'int'),
 ('week', 'int'),
 ('month', 'int'),
 ('year', 'int'),
 ('weekday', 'int')]

In [39]:
# Sending time table partition by year & month
time_table.write.parquet('./output/time.parquet', mode='overwrite', partitionBy=['year', 'month'])

> **2. Users_table creation from 'log_df' table**

In [40]:
# creating users table with columns user_id, first_name, last_name, gender, level
users_table = log_df.select(['userId', 'firstName', 'lastName', 'gender', 'level'])\
        .withColumnRenamed('userId', 'user_id')\
        .withColumnRenamed('firstName', 'first_name')\
        .withColumnRenamed('lastName', 'last_name').dropDuplicates()

users_table.toPandas().head(3)

,user_id,first_name,last_name,gender,level
0,26,Ryan,Smith,M,free
1,7,Adelyn,Jordan,F,free
2,71,Ayleen,Wise,F,free


In [41]:
# sending user_table to a parquet file without any partition 
users_table.write.parquet('./output/users.parquet', mode='overwrite')

> **3. songs_table creation from 'song_df' table**

In [42]:
# creating songs table with columns song_id, title, artist_id, year, duration
songs_table = song_df.select(['song_id', 'artist_id', 'title', 'year', 'duration']).dropDuplicates()
songs_table.toPandas().head(3)

,song_id,artist_id,title,year,duration
0,SOHUOAP12A8AE488E9,ARD842G1187B997376,Floating,1987,491.12771
1,SOKEJEJ12A8C13E0D0,ARC43071187B990240,The Urgency (LP Version),0,245.21098
2,SONHOTT12A8C13493C,AR7G5I41187FB4CE6C,Something Girls,1982,233.40363


In [43]:
# partition by year & artist, since songs_table contains artist_id, 
# so instead of artist_name, artist_id is used.
songs_table.write.parquet('./output/songs.parquet', mode='overwrite', partitionBy=['year', 'artist_id'])

> **4. artists_table creation from 'song_df' table**

In [44]:
# creating artists table with columns artist_id, name, location, latitude, longitude
artists_table = song_df.select(['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'])\
                .withColumnRenamed('artist_name','name') \
                .withColumnRenamed('artist_location','location') \
                .withColumnRenamed('artist_latitude','latitude') \
                .withColumnRenamed('artist_longitude','longitude').dropDuplicates()

artists_table.toPandas().head(3)

,artist_id,name,location,latitude,longitude
0,ARXR32B1187FB57099,Gob,,NaN,NaN
1,AROGWRA122988FEE45,Christos Dantis,,NaN,NaN
2,AREDL271187FB40F44,Soul Mekanik,,NaN,NaN


In [45]:
# sending artist_table to 'parquet' file without any partition
artists_table.write.parquet('./output/artists.parquet', mode='overwrite')

In [46]:
artists_table.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)



In [47]:
log_df.dtypes

[('artist', 'string'),
 ('auth', 'string'),
 ('firstName', 'string'),
 ('gender', 'string'),
 ('itemInSession', 'bigint'),
 ('lastName', 'string'),
 ('length', 'double'),
 ('level', 'string'),
 ('location', 'string'),
 ('method', 'string'),
 ('page', 'string'),
 ('registration', 'double'),
 ('sessionId', 'bigint'),
 ('song', 'string'),
 ('status', 'bigint'),
 ('ts', 'bigint'),
 ('userAgent', 'string'),
 ('userId', 'string'),
 ('start_time', 'timestamp'),
 ('day', 'int'),
 ('month', 'int'),
 ('year', 'int'),
 ('hour', 'int'),
 ('minute', 'int'),
 ('second', 'int'),
 ('week', 'int'),
 ('weekday', 'int')]

In [48]:
song_df.dtypes

[('artist_id', 'string'),
 ('artist_latitude', 'float'),
 ('artist_longitude', 'float'),
 ('artist_location', 'string'),
 ('artist_name', 'string'),
 ('song_id', 'string'),
 ('title', 'string'),
 ('duration', 'double'),
 ('year', 'int')]

> **Joining tables for songplay tables ( log_df + song_df )table**

In [49]:
# creating 'songplays_table' by joining with'song_df' table
songplays_table = log_df.join(songs_table, log_df.song == songs_table.title, how='inner').dropDuplicates() \
                        .select(monotonically_increasing_id().alias("songplay_id"), \
                                col("start_time"), \
                                col("userId").alias("user_id"), \
                                "level","song_id","artist_id", \
                                col("sessionId").alias("session_id"), \
                                "location", \
                                col("userAgent").alias("user_agent"))

songplays_table.toPandas().head(3)

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,386547056640,2018-11-19 09:14:20.796,24,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,672,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"""
1,678604832768,2018-11-21 21:56:47.796,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
2,1400159338496,2018-11-14 05:06:03.796,10,free,SOGDBUF12A8C140FAA,AR558FS1187FB45658,484,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"""


In [50]:
# These are 3 joins
# creating 'songplays_table' by joining with'song_df' table
#songplays_table = log_df.join(song_df, (log_df.song == song_df.title) \
#                              & (log_df.artist == song_df.artist_name), how='inner').dropDuplicates()\
#                                         .select(monotonically_increasing_id().alias("songplay_id"), \
#                                col("start_time"), \
#                                col("userId").alias("user_id"), \
#                                "level","song_id","artist_id", \
#                                col("sessionId").alias("session_id"), \
#                                "location", \
#                                col("userAgent").alias("user_agent"))

# songplays_table.toPandas().head(3)

In [51]:
# These are 2 joins
# creating 'songplays_table' by joining with'song_df' table
#songplays_table = log_df.join(song_df, (log_df.song == song_df.title) \
#                              & (log_df.artist == song_df.artist_name)\
#                              & (log_df.length == song_df.duration), how='inner').dropDuplicates()\
#                                         .select(monotonically_increasing_id().alias("songplay_id"), \
#                                col("start_time"), \
#                                col("userId").alias("user_id"), \
#                                "level","song_id","artist_id", \
#                                col("sessionId").alias("session_id"), \
#                                "location", \
#                                col("userAgent").alias("user_agent"))

#songplays_table.toPandas().head(3)

In [52]:
songplays_table.printSchema()

root
 |-- songplay_id: long (nullable = false)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [53]:
print(songplays_table.toPandas().shape)

(4, 9)


> **Joining tables for songplay tables ( songplays_table + time_table )**

In [54]:
# Joining 'songplay_table' and 'time_table'
songplays_table = songplays_table.join(time_table, songplays_table.start_time == time_table.start_time, how="inner") \
                        .select("songplay_id", songplays_table.start_time, "user_id", "level", "song_id", \
                                "artist_id", "session_id", "location", "user_agent", "year", "month").drop_duplicates()

In [55]:
songplays_table.toPandas().head(3)

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent,year,month
0,1400159338496,2018-11-14 05:06:03.796,10,free,SOGDBUF12A8C140FAA,AR558FS1187FB45658,484,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4""",2018,11
1,1408749273088,2018-11-27 22:35:59.796,80,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,992,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36""",2018,11
2,386547056640,2018-11-19 09:14:20.796,24,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,672,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36""",2018,11


In [56]:
songplays_table.printSchema()

root
 |-- songplay_id: long (nullable = false)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [57]:
print(songplays_table.toPandas().shape)

(4, 11)


In [58]:
songplays_table.describe()

DataFrame[summary: string, songplay_id: string, user_id: string, level: string, song_id: string, artist_id: string, session_id: string, location: string, user_agent: string, year: string, month: string]

In [59]:
songplays_table.count()

4

### `QUERIES: `

In [60]:
songplays_table.createOrReplaceTempView('songplays')
songs_table.createOrReplaceTempView('songs')
users_table.createOrReplaceTempView('users')
artists_table.createOrReplaceTempView('artists')
time_table.createOrReplaceTempView('time')

In [61]:
# 1. Basic song-user-artist view
songplays_query = spark.sql("""
SELECT sp.user_id, sp.songplay_id, st.song_id, at.artist_id, st.title 
FROM songplays sp 
INNER JOIN songs st ON (sp.song_id = st.song_id)
INNER JOIN artists at ON (sp.artist_id = at.artist_id)
""")
songplays_query.show(3, truncate=False)

+-------+-------------+------------------+------------------+-----+
|user_id|songplay_id  |song_id           |artist_id         |title|
+-------+-------------+------------------+------------------+-----+
|10     |1400159338496|SOGDBUF12A8C140FAA|AR558FS1187FB45658|Intro|
|80     |1408749273088|SOGDBUF12A8C140FAA|AR558FS1187FB45658|Intro|
|24     |386547056640 |SOGDBUF12A8C140FAA|AR558FS1187FB45658|Intro|
+-------+-------------+------------------+------------------+-----+
only showing top 3 rows



In [62]:
# 2. number of song listened by users
num_song_by_user = spark.sql(
"""
SELECT DISTINCT (songplays.user_id), users.first_name, users.last_name, COUNT(songplays.song_id) AS num_songs_listened\
     FROM songplays\
     INNER JOIN users ON (songplays.user_id = users.user_id)\
     INNER JOIN songs ON (songplays.song_id = songs.song_id)\
     GROUP BY songplays.user_id, users.first_name, users.last_name\
     ORDER BY num_songs_listened DESC
""")

num_song_by_user.toPandas().head()

,user_id,first_name,last_name,num_songs_listened
0,15,Lily,Koch,2
1,80,Tegan,Levine,2
2,10,Sylvie,Cruz,1
3,24,Layla,Griffin,1


In [69]:
# 3. number of hour spent by users listening songs
hours_spent_listening = spark.sql(
"""
SELECT DISTINCT (songplays.user_id), users.first_name, users.last_name, AVG(time.hour) AS avg_hours_spent\
    FROM songplays\
    JOIN users ON (songplays.user_id = users.user_id)\
    JOIN time ON (songplays.start_time = time.start_time)\
    GROUP BY songplays.user_id, users.first_name, users.last_name\
    ORDER BY avg_hours_spent DESC
""")

hours_spent_listening.toPandas().head(4)

,user_id,first_name,last_name,avg_hours_spent
0,80,Tegan,Levine,22.0
1,15,Lily,Koch,21.0
2,24,Layla,Griffin,9.0
3,10,Sylvie,Cruz,5.0


In [64]:
# most listened artists
most_listened_artist = spark.sql(
"""
SELECT songs.title AS song_title, artists.name, COUNT(songplays.song_id) AS num_listened
     FROM songplays
     JOIN songs ON (songplays.song_id = songs.song_id)
     JOIN artists ON (songplays.artist_id = artists.artist_id)
     GROUP BY songs.title, artists.name
     ORDER BY num_listened DESC
""")

most_listened_artist.toPandas().head(3)

,song_title,name,num_listened
0,Intro,40 Grit,3
1,Setanta matins,Elena,1


In [65]:
new_query2 = spark.sql("""
SELECT users.user_id, COUNT(songplays.song_id) AS num_song
FROM users 
INNER JOIN songplays  ON songplays.user_id = users.user_id
INNER JOIN songs  ON songplays.song_id = songs.song_id
GROUP BY users.user_id
""")

new_query2.show(2)

+-------+--------+
|user_id|num_song|
+-------+--------+
|     15|       2|
|     24|       1|
+-------+--------+
only showing top 2 rows



In [66]:
users_query = spark.sql(
"""
SELECT songplays.user_id, SUM(time.hour) AS total_hour
FROM songplays 
JOIN time 
ON songplays.start_time = time.start_time
GROUP BY songplays.user_id
ORDER BY total_hour DESC
""")

users_query.show(3)

+-------+----------+
|user_id|total_hour|
+-------+----------+
|     80|        22|
|     15|        21|
|     24|         9|
+-------+----------+
only showing top 3 rows

